## API + Bib stuff

In [2]:
#from pandas_profiling import ProfileReport
import sqlite3
import pandas as pd
from collections import Counter
import re
import multiprocessing as mp
from langchain.document_loaders import BSHTMLLoader, DirectoryLoader
import os
import openai
import os
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import pinecone

import pinecone
from langchain.vectorstores import Pinecone

/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)


## load and preprocessing

In [17]:
conn = sqlite3.connect('./all-the-news.db')
df = pd.read_sql_query("SELECT * FROM longform", conn)
conn.close()

In [18]:
df.size

2449620

In [7]:
df.head(10)

,id,title,author,date,content,year,month,publication,category,digital,section,url
0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017.0,5.0,Verge,Longform,1.0,None,None
1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017.0,5.0,Verge,Longform,1.0,None,None
2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017.0,5.0,Verge,Longform,1.0,None,None
3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017.0,5.0,Verge,Longform,1.0,None,None
4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017.0,5.0,Verge,Longform,1.0,None,None
5,6,Instant recall,\nCasey Newton\n,2017-04-16,Facebook's Instant Articles promised to ...,2017.0,4.0,Verge,Longform,1.0,None,None
6,7,Massive attack,\nColin Lecher\n,2017-04-14,How a weapon against war became a weapon...,2017.0,4.0,Verge,Longform,1.0,None,None
7,8,The Future Agency,\nKyle Chayka\n,2017-03-30,Inside the big business of imagining the...,2017.0,3.0,Verge,Longform,1.0,None,None
8,9,Flight Risk,\nBen Popper\n,2017-03-28,Legal threats and disgruntled clients: i...,2017.0,3.0,Verge,Longform,1.0,None,None
9,10,Brain drain,\nCasey Newton\n,2017-03-15,Genius quietly laid off a bunch of its e...,2017.0,3.0,Verge,Longform,1.0,None,None


In [5]:
titles = df['title'].tolist()
contents = df['content'].tolist()

In [6]:

def preprocess_text(text):
    if text is not None:
        text = text.lower()  # Konvertiere in Kleinbuchstaben
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Entferne Sonderzeichen
    return text

processed_titles = [preprocess_text(title) for title in titles]
processed_contents = [preprocess_text(content) for content in contents]


In [9]:
#erstelle neues df mit den prozessierten titeln und contents und den originalen titeln und contents und der id
df_processed = pd.DataFrame({'title': processed_titles, 'content': processed_contents, 'original_title': titles, 'original_content': contents, 'id': df['id']})




In [10]:
df_processed.head(10)

,title,content,original_title,original_content,id
0,agent cooper in twin peaks is the audience onc...,and never more so than in showtimes new ...,Agent Cooper in Twin Peaks is the audience: on...,And never more so than in Showtime’s new...,1
1,ai the humanity,alphagos victory isnt a defeat for human...,"AI, the humanity!",AlphaGo’s victory isn’t a defeat for hum...,2
2,the viral machine,super deluxe built a weird internet empi...,The Viral Machine,Super Deluxe built a weird internet empi...,3
3,how anker is beating apple and samsung at thei...,steven yang quit his job at google in th...,How Anker is beating Apple and Samsung at thei...,Steven Yang quit his job at Google in th...,4
4,tour black panthers reimagined homeland with t...,ahead of black panthers 2018 theatrical ...,Tour Black Panther’s reimagined homeland with ...,Ahead of Black Panther’s 2018 theatrical...,5
5,instant recall,facebooks instant articles promised to t...,Instant recall,Facebook's Instant Articles promised to ...,6
6,massive attack,how a weapon against war became a weapon...,Massive attack,How a weapon against war became a weapon...,7
7,the future agency,inside the big business of imagining the...,The Future Agency,Inside the big business of imagining the...,8
8,flight risk,legal threats and disgruntled clients in...,Flight Risk,Legal threats and disgruntled clients: i...,9
9,brain drain,genius quietly laid off a bunch of its e...,Brain drain,Genius quietly laid off a bunch of its e...,10


In [11]:
df_processed.dropna()

,title,content,original_title,original_content,id
0,agent cooper in twin peaks is the audience onc...,and never more so than in showtimes new ...,Agent Cooper in Twin Peaks is the audience: on...,And never more so than in Showtime’s new...,1
1,ai the humanity,alphagos victory isnt a defeat for human...,"AI, the humanity!",AlphaGo’s victory isn’t a defeat for hum...,2
2,the viral machine,super deluxe built a weird internet empi...,The Viral Machine,Super Deluxe built a weird internet empi...,3
3,how anker is beating apple and samsung at thei...,steven yang quit his job at google in th...,How Anker is beating Apple and Samsung at thei...,Steven Yang quit his job at Google in th...,4
4,tour black panthers reimagined homeland with t...,ahead of black panthers 2018 theatrical ...,Tour Black Panther’s reimagined homeland with ...,Ahead of Black Panther’s 2018 theatrical...,5
...,...,...,...,...,...
67359,indiana governor mike pence endorses ted cruz,i want to receive updates from partners and s...,Indiana Governor Mike Pence Endorses Ted Cruz,"[, I want to receive updates from partners and...",77036
67360,caught doping,the main site for archive team is at archivete...,Caught Doping,[The main site for Archive Team is at archivet...,77037
67361,the business of the dnc convention,i want to receive updates from partners and s...,The Business of the DNC Convention,"[, I want to receive updates from partners and...",77038
67362,you cant escape data surveillance in america,i want to receive updates from partners and s...,You Can't Escape Data Surveillance In America,"[, I want to receive updates from partners and...",77039


In [21]:
#new_df = pd.DataFrame({'title_proc': processed_titles, 'content_proc': processed_contents})
#print(new_df.size)
#new_df = new_df.dropna()
#print(new_df.size)

408270
382362


In [13]:
# Definieren Sie eine Liste der zu entfernenden Wörter
remove_words = ['the', 'to', 'of', 'a', 'and', 'in', 'that', 's', 'is', 'for', 
                'on', 'was', 'it', 'with', 'The', 'I', 'as', 'he', 'said', 'at', 
                'his', 'have', 'be', 'by', 'from', 'are', 'has', 'not', 'an']

# Definieren Sie eine Funktion, um die Wörter zu entfernen
def remove_specific_words(text):
    return ' '.join([word for word in text.split() if word not in remove_words])

# Wenden Sie die Funktion auf die Spalte an
df_processed['content'] = df_processed['content'].apply(remove_specific_words)


In [14]:
df_processed.isna().sum()

title               0
content             0
original_title      0
original_content    0
id                  0
dtype: int64

In [19]:
print(df_processed['content'].head())
print(df_processed['original_content'].head())

0    never more so than showtimes new series reviva...
1    alphagos victory isnt defeat humans its opport...
2    super deluxe built weird internet empire can s...
3    steven yang quit job google summer 2011 build ...
4    ahead black panthers 2018 theatrical release m...
Name: content, dtype: object
0          And never more so than in Showtime’s new...
1          AlphaGo’s victory isn’t a defeat for hum...
2          Super Deluxe built a weird internet empi...
3          Steven Yang quit his job at Google in th...
4          Ahead of Black Panther’s 2018 theatrical...
Name: original_content, dtype: object


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Laden Sie die Stopwords herunter und initialisieren Sie den Lemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenisierung
    words = word_tokenize(text)
    
    # Entfernen der Stoppwörter und Lemmatisierung
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(words)




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df_processed['content'] = df_processed['content'].fillna('')
df_processed['content'] = df_processed['content'].apply(preprocess_text)

In [25]:
print(df_processed['content'].head())
print(df_processed['original_content'].head())

0    never showtime new series revival spoiler ahea...
1    alphagos victory isnt defeat human opportunity...
2    super deluxe built weird internet empire succe...
3    steven yang quit job google summer 2011 build ...
4    ahead black panther 2018 theatrical release ma...
Name: content, dtype: object
0          And never more so than in Showtime’s new...
1          AlphaGo’s victory isn’t a defeat for hum...
2          Super Deluxe built a weird internet empi...
3          Steven Yang quit his job at Google in th...
4          Ahead of Black Panther’s 2018 theatrical...
Name: original_content, dtype: object


## Split into Chunks

In [26]:

tokenizer = tiktoken.get_encoding("cl100k_base")
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [68]:
df_processed.head()

,title,content,original_title,original_content,id
0,agent cooper in twin peaks is the audience onc...,never showtime new series revival spoiler ahea...,Agent Cooper in Twin Peaks is the audience: on...,And never more so than in Showtime’s new...,1
1,ai the humanity,alphagos victory isnt defeat human opportunity...,"AI, the humanity!",AlphaGo’s victory isn’t a defeat for hum...,2
2,the viral machine,super deluxe built weird internet empire succe...,The Viral Machine,Super Deluxe built a weird internet empi...,3
3,how anker is beating apple and samsung at thei...,steven yang quit job google summer 2011 build ...,How Anker is beating Apple and Samsung at thei...,Steven Yang quit his job at Google in th...,4
4,tour black panthers reimagined homeland with t...,ahead black panther 2018 theatrical release ma...,Tour Black Panther’s reimagined homeland with ...,Ahead of Black Panther’s 2018 theatrical...,5


In [71]:
from typing_extensions import Concatenate
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

# loop over DataFrame rows
for idx, row in tqdm(df_processed.iterrows(), total=df_processed.shape[0]):
    title = row['original_title']
    o_content = row['original_content']
    content = row['content']
    id = row['id']
    if len(content) > 100:
        texts = text_splitter.split_text(content)
        chunks.extend([{
            'id': str(uuid4()),
            'o_id': id,
            'title': title,
            'original_content': o_content,
            'text': texts[i],
            'chunk': i,

        } for i in range(len(texts))])

100%|██████████| 67364/67364 [02:41<00:00, 416.71it/s] 


In [72]:
chunks_backup = chunks

In [30]:
df_processed.head(5)

,title,content,original_title,original_content,id
0,agent cooper in twin peaks is the audience onc...,never showtime new series revival spoiler ahea...,Agent Cooper in Twin Peaks is the audience: on...,And never more so than in Showtime’s new...,1
1,ai the humanity,alphagos victory isnt defeat human opportunity...,"AI, the humanity!",AlphaGo’s victory isn’t a defeat for hum...,2
2,the viral machine,super deluxe built weird internet empire succe...,The Viral Machine,Super Deluxe built a weird internet empi...,3
3,how anker is beating apple and samsung at thei...,steven yang quit job google summer 2011 build ...,How Anker is beating Apple and Samsung at thei...,Steven Yang quit his job at Google in th...,4
4,tour black panthers reimagined homeland with t...,ahead black panther 2018 theatrical release ma...,Tour Black Panther’s reimagined homeland with ...,Ahead of Black Panther’s 2018 theatrical...,5


In [73]:
#dispaly the first 5 chunks
chunks[:5]

[{'id': '8f53c18d-cc81-450a-95c7-b690aabe0535',
  'o_id': 1,
  'title': 'Agent Cooper in Twin Peaks is the audience: once delighted, now disintegrating',
  'original_content': '      And never more so than in Showtime’s new series revival Some spoilers ahead through episode 4 of season 3 of Twin Peaks. On May 21st, Showtime brought back David Lynch’s groundbreaking TV series Twin Peaks, and fulfilled a prophecy in the process. In the second season finale, back in 1991, the spirit of series-defining murder victim Laura Palmer told FBI special agent and series protagonist Dale Cooper, “I’ll see you again in 25 years.” That clip plays again in the first episode of Lynch’s Twin Peaks revival, as a reminder that decades have in fact gone by, Laura’s promise has been carried out, and a series canceled mid-story is back on the air.A lot has changed in 25 years. The original cast members, who are mostly back on board, have all aged heavily and visibly. Many of the characters have moved on in l

## Generate Embedding

In [32]:

chunks[1000], chunks[10000], chunks[20000]


({'id': 'c163c66e-32f8-4d0b-ba2a-4834b46628c5',
  'title': 'theranos proprietary tech wasnt vetted by federal inspectors for two years',
  'text': 'theranos silicon valley blood testing company launched first bloodsampling center end 2013 two year followed federal official didnt inspect theranos invention verge learned suggests one two possibility either theranos customer received test result generated equipment federal official checked accuracy theranos wasnt using proprietary technology behind 9 billion valuation return result patient recent report wall street journal pointed toward extreme version option number one theranos actively hid proprietary technology lab inspector would explain invention never checked theranos publicly proclaimed used technology patient mean either company lying patient received test validated federal authority steve jurvetson flickrcms inspect theranos proprietary technologytheranos first came scrutiny report journal october theranos invention used small n

In [74]:
import torch
if torch.cuda.is_available():
    print("GPU ist verfügbar")
    device = torch.device('cuda')
else:
    print("GPU ist nicht verfügbar")
    device = torch.device('cpu')

GPU ist nicht verfügbar


In [140]:
from tqdm import tqdm
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

def text_to_embedding(text):
    return embeddings.embed_query(text)

for chunk in tqdm(chunks, desc='Vektorisierung des Textes'):
    chunk['embedding'] = text_to_embedding(chunk['text'])


Vektorisierung des Textes: 100%|██████████| 989/989 [05:17<00:00,  3.11it/s]


## Pinecone

In [67]:
chunks[0]

{'id': 'd366926e-333a-4f0e-9744-0027761ff88f',
 'title': 'agent cooper in twin peaks is the audience once delighted now disintegrating',
 'text': 'never showtime new series revival spoiler ahead episode 4 season 3 twin peak may 21st showtime brought back david lynchs groundbreaking tv series twin peak fulfilled prophecy process second season finale back 1991 spirit seriesdefining murder victim laura palmer told fbi special agent series protagonist dale cooper ill see 25 year clip play first episode lynchs twin peak revival reminder decade fact gone lauras promise carried series canceled midstory back aira lot changed 25 year original cast member mostly back board aged heavily visibly many character moved life getting new job forming family taking new obsession opening episode dale cooper still show left 1991 trapped spirit domain known black lodge mercy incomprehensible force behave erratic alien way word he like anyone who actually watching twin peak third season began audience also s

In [142]:
#safe chunks to json
import json
with open('chunksmini.json', 'w') as fp:
    json.dump(chunks, fp)



In [41]:
#load chunks from json
import json
with open('chunksmini.json', 'r') as fp:
    chunks1 = json.load(fp)
    



In [144]:
vector_dimension = len(chunks[0]['embedding'])
print(f"The dimension of the data is {vector_dimension}")
vector_dimension = len(chunks[100]['embedding'])
print(f"The dimension of the data is {vector_dimension}")


The dimension of the data is 768
The dimension of the data is 768


In [159]:
chunks[0]

{'id': 'b584db2a-d93d-4f10-a3a3-ac4792c2edd0',
 'title': 'agent cooper in twin peaks is the audience once delighted now disintegrating',
 'text': 'never showtime new series revival spoiler ahead episode 4 season 3 twin peak may 21st showtime brought back david lynchs groundbreaking tv series twin peak fulfilled prophecy process second season finale back 1991 spirit seriesdefining murder victim laura palmer told fbi special agent series protagonist dale cooper ill see 25 year clip play first episode lynchs twin peak revival reminder decade fact gone lauras promise carried series canceled midstory back aira lot changed 25 year original cast member mostly back board aged heavily visibly many character moved life getting new job forming family taking new obsession opening episode dale cooper still show left 1991 trapped spirit domain known black lodge mercy incomprehensible force behave erratic alien way word he like anyone who actually watching twin peak third season began audience also s

In [13]:
index_name = 'langchain-retrieval-augmentation-full'

In [ ]:
index_name = "my-new-index"
pinecone.create_index(index_name, metric="cosine", shards=1, dimension=vector_dimension)#768


In [160]:
texts = [chunk['text'] for chunk in chunks]

In [179]:
#make a simularities search on the pinecone index
def search_similar_texts(text, index_name, top_k=5):
    query_embedding = text_to_embedding(text)
    results = pinecone.query(index_name, query_embedding, top_k=top_k)
    return results.ids, results.distances



In [185]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 989}},
 'total_vector_count': 989}

In [ ]:
search_similar_texts("texas", index_name)

In [ ]:
vstore = Pinecone.from_texts(texts, embeddings, index_name='my-new-index1')

In [15]:
# connect to index
index = pinecone.Index(index_name=index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.8,
 'namespaces': {'': {'vector_count': 98025}},
 'total_vector_count': 98025}

In [153]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)

    embeds = [x['embedding'] for x in meta_batch]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

100%|██████████| 10/10 [00:47<00:00,  4.77s/it]


In [ ]:
from tqdm.auto import tqdm

batch_size = 100  # wie viele Einbettungen wir auf einmal erstellen und einfügen

for i in tqdm(range(0, len(chunks), batch_size)):
    # finde das Ende des Batches
    i_end = min(len(chunks), i+batch_size)
    chunk_batch = chunks[i:i_end]
    
    # hole die IDs und die Einbettungen
    ids_batch = [chunk['id'] for chunk in chunk_batch]
    embeddings_batch = [chunk['embedding'] for chunk in chunk_batch]
    
    # Erstelle eine Liste von Tupeln zum Hochladen
    to_upsert = list(zip(ids_batch, embeddings_batch))
    print(to_upsert)
    
    # Lade die Daten in Pinecone hoch
    index.upsert(vectors=to_upsert)


In [168]:
query_text = "texas"
# Wandeln Sie den Abfragetext in die gleiche Form der Einbettungen um, die in Pinecone gespeichert sind
query_embedding = embeddings.embed_query(query_text)

# Führen Sie die Abfrage durch
results = index.query(queries=[query_embedding], top_k=5)

# Drucken Sie die Ergebnisse aus
for i, item in enumerate(results.results):
    print(f"Rank {i+1}, id: {item.id}, score: {item.score}")


Rank 1, id: None, score: None


In [171]:
vstore

In [197]:
query = "Who is Patrick Ries"
result = vstore.similarity_search(query)
print(result)

[Document(page_content='second sunday april year michael c ruppert broke 63yearold copturnedwriter firebrand gained fame starring collapse 2009 documentary predicts society destruction publicity film great went countrywide promotional tour compensation fizzled april receiving couple hundred dollar per month royalty supplement meager social security checksin effort simplify continue readingamphellip', metadata={}), Document(page_content='second sunday april year michael c ruppert broke 63yearold copturnedwriter firebrand gained fame starring collapse 2009 documentary predicts society destruction publicity film great went countrywide promotional tour compensation fizzled april receiving couple hundred dollar per month royalty supplement meager social security checksin effort simplify continue readingamphellip', metadata={}), Document(page_content='called carnationsbut also head business martin formed part musicbooking agency triangle talent worked major band area began pick engineering s

In [ ]:
from tqdm.auto import tqdm

batch_size = 100  # wie viele Einbettungen wir auf einmal erstellen und einfügen

for i in tqdm(range(0, len(chunks), batch_size)):
    # finde das Ende des Batches
    i_end = min(len(chunks), i+batch_size)
    chunk_batch = chunks[i:i_end]
    
    # hole die IDs und die Einbettungen
    ids_batch = [chunk['id'] for chunk in chunk_batch]
    embeddings_batch = [chunk['embedding'] for chunk in chunk_batch]
    
    # Erstelle eine Liste von Tupeln zum Hochladen
    to_upsert = list(zip(ids_batch, embeddings_batch))
    
    # Lade die Daten in Pinecone hoch
    index.upsert(vectors=to_upsert)


In [66]:
len(embeddings.embed_query(query_text))

768

In [149]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def create_embedding(query):
    return embeddings.embed_query(query)

query = "david lynchs"
query_embedding = create_embedding(query)

# Nehmen Sie das erste Element in Ihren Chunks
first_chunk_embedding = chunks[0]['embedding']

# Sklearn erwartet 2D-Arrays, also fügen wir eine zusätzliche Dimension hinzu
query_embedding = np.array(query_embedding)[np.newaxis, :]
first_chunk_embedding = np.array(first_chunk_embedding)[np.newaxis, :]

similarity = cosine_similarity(query_embedding, first_chunk_embedding)

print(f"Die Cosinus-Ähnlichkeit zwischen der Anfrage und dem ersten Chunk beträgt {similarity[0][0]}")


Die Cosinus-Ähnlichkeit zwischen der Anfrage und dem ersten Chunk beträgt 0.35650228711247944


In [ ]:
def query_pinecone_embedding(embedding):
    # Rufen Sie von Pinecone ab
    print(len(embedding))  # Diese Zeile ist nur zum Debuggen, sie druckt die Länge der Anfrageeinbettung
    # Holen Sie sich relevante Kontexte (einschließlich der Fragen)
    return index.query(vectors=[embedding], top_k=5, include_metadata=True)

def create_embedding(query):
    return embeddings.embed_query(query)

query = "What is pharmaceuticals international?"

embedding = create_embedding(query)
result = query_pinecone_embedding(embedding)
print(result)


In [ ]:
#zeige mir die ersten 5 chunks_backup
chunks[:5]

In [ ]:
#make a vectorstore from pinecone


## Kapitel OpenAI

In [39]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [36]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")

26

In [43]:
tiktoken_len(chunks[2]['text'])

424

In [57]:
import os
from dotenv import load_dotenv
load_dotenv(".env", override=True)

True

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

In [62]:
index_name = 'langchain-retrieval-augmentation-full'

In [39]:
res = embed.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

In [65]:
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=len(res[0])  # 1536 dim of text-embedding-ada-002
    )

In [75]:
# connect to index
index = pinecone.Index(index_name=index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [76]:
chunks[0]

{'id': '8f53c18d-cc81-450a-95c7-b690aabe0535',
 'o_id': 1,
 'title': 'Agent Cooper in Twin Peaks is the audience: once delighted, now disintegrating',
 'original_content': '      And never more so than in Showtime’s new series revival Some spoilers ahead through episode 4 of season 3 of Twin Peaks. On May 21st, Showtime brought back David Lynch’s groundbreaking TV series Twin Peaks, and fulfilled a prophecy in the process. In the second season finale, back in 1991, the spirit of series-defining murder victim Laura Palmer told FBI special agent and series protagonist Dale Cooper, “I’ll see you again in 25 years.” That clip plays again in the first episode of Lynch’s Twin Peaks revival, as a reminder that decades have in fact gone by, Laura’s promise has been carried out, and a series canceled mid-story is back on the air.A lot has changed in 25 years. The original cast members, who are mostly back on board, have all aged heavily and visibly. Many of the characters have moved on in life,

In [55]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=model_name)
    except Exception as e:
        print(e)
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=model_name)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'title': x['title'],
        'o_id': x['o_id'],
        'original_content': x['original_content'],
        'chunk': x['chunk'],
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 1/3161 [00:07<6:54:13,  7.87s/it]


ApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Sun, 11 Jun 2023 03:02:35 GMT', 'x-envoy-upstream-service-time': '9', 'content-length': '70', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"index is full, cannot accept data.","details":[]}


In [192]:
def create_embedding(query):
    return openai.Embedding.create(
        input=[query],
        engine=model_name
    )

def query_pinecone_embedding(openai_embedding):
    # retrieve from Pinecone
    xq = openai_embedding['data'][0]['embedding']
    # get relevant contexts (including the questions)
    return index.query(xq, top_k=8, include_metadata=True)

In [52]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [54]:
query = "Where is Texas?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='rio grande city texas texas state trooper shot patrolling texas borderstarr county official confirmed breitbart texas trooper apparently struck stray bullet gun battle taking place across mexican border shooting took place near rom texas area immediately north miguel aleman advertisement trooper airlifted hospital mcallen expected recover update 315pm local time law enforcement official confirmed breitbart texas u border patrol agent shot well dutyissued ballistic gear protected federal state authority looking case determine shot fact stray bullet gun battle mexican side purposely aimed law enforcement texas department public safety sgt johnny hernandez confirmed breitbart texas trooper shot leg undergoing test expected recover breitbart texas provide update become available ildefonso ortiz award winning journalist breitbart texas cofounded cartel chronicle project follow twitter facebook', metadata={'chunk': 0.0, 'title': 'texas state trooper border patrol agen

In [5]:
def create_embedding(query):
    return openai.Embedding.create(
        input=[query],
        engine=model_name
    )

In [7]:
def query_pinecone_embedding(openai_embedding):
    # retrieve from Pinecone
    xq = openai_embedding['data'][0]['embedding']
    print(len(xq))
    # get relevant contexts (including the questions)
    return index.query(xq, top_k=15, include_metadata=True)

In [16]:
query = "Where is Luxemburg?"

embedding = create_embedding(query)
print(len(embedding))
result = query_pinecone_embedding(embedding)
result

4
1536


{'matches': [{'id': '54af59b8-53d7-46ee-9c8d-d1f41159b8fa',
              'metadata': {'chunk': 0.0,
                           'o_id': 6864.0,
                           'title': 'Frank Gehry drops a spaceship in Paris'},
              'score': 0.804197609,
              'values': []},
             {'id': 'e7955eac-6913-4898-8c2f-0069f1bca0a3',
              'metadata': {'chunk': 1.0,
                           'o_id': 25806.0,
                           'title': 'After ‘Brexit,’ Finding a New London for '
                                    'the Financial World to Call Home - The '
                                    'New York Times'},
              'score': 0.795961618,
              'values': []},
             {'id': 'afd9e8e2-00ab-4d83-8c96-565d99cada55',
              'metadata': {'chunk': 1.0,
                           'o_id': 47889.0,
                           'title': 'Brexit: How Europe Fell Out Of Love With '
                                    'The EU'},
              'sc

In [9]:
model="gpt-3.5-turbo"

In [99]:
def query_gpt(prompt, message, model):
     return openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": message}
        ]
    )

In [132]:
def pinecone_to_list(res):
    # get list of retrieved texts
    contexts = [item['metadata']['text'] for item in res['matches']]
    
    # get list of metadata titles
    titles = [item['metadata']['title'] for item in res['matches']]

    items = ["Title = " + title + "\nContent = " + context for title, context in zip(titles, contexts)]
    
    return "\n\n---\n\n".join(items)+"\n\n-----\n\n"+query


In [269]:
prompt = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question and references the Source. 
If the information can not be found in the information
provided by the user you truthfully say "I don't know".
For every Answer you provide you need to reference a Source out of the information provided by the user.
If you can not find a source you truthfully say "I don't know".
"""

In [104]:
def search_docs(query, model):
    embedding = create_embedding(query)
    result = query_pinecone_embedding(embedding)
    open_ai_message = pinecone_to_list(result)
    return query_gpt(prompt, open_ai_message, model=model)

In [125]:
def search_docs(query, model):
    embedding = create_embedding(query)
    result = query_pinecone_embedding(embedding)
    open_ai_message = pinecone_to_list(result)
    print(open_ai_message)
    return query_gpt(prompt, open_ai_message, model=model)

In [ ]:
query = "List all Articles about Trump"
result = search_docs(query, model=model)

In [276]:
from IPython.display import Markdown

display(Markdown(result['choices'][0]['message']['content']))

1. "Apples Most Important Introductions: Tim Cook, Craig Federighi, and Jony Ive"
2. "Apple Developer Conference: A More Open Siri and Other Upgrades" 
3. "Apple Announced the Future This Week and You Probably Didn't Even Notice" 
4. "Apple CEO Tim Cook's Pay Slumps Along with iPhone Sales" 
5. "Apple Is Losing Its Focus Again and This Time There's No Steve Jobs Coming to the Rescue" 
6. "There's a Bull Case for Apple, but the Company Won't Tell You About It" 
7. "Here's Everything We Know About the iPhone 7 So Far" 
8. "By Buying Ireland-Based Allergan, Pfizer Could Avoid Higher US Rates" 

Articles 1-5 and 7 are about Apple Inc., while Articles 6 and 8 only mention Apple in passing but are not specifically about the company.